# LAX normal temperatures
> This notebook fetches and process data about normal mean, max and min temperatures. The data come from [SERCC](https://sercc.oasis.unc.edu/Map.php?region=conus).

---

#### Import Python tools and Jupyter config

In [1]:
import pandas as pd
import jupyter_black
import altair as alt
from altair_stiles import theme
import geopandas as gpd
from tqdm.notebook import tqdm

In [2]:
alt.themes.register("altair_stiles", theme)
alt.themes.enable("altair_stiles")

ThemeRegistry.enable('altair_stiles')

In [3]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [4]:
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

In [5]:
# Read data and filter for LAX
lax = "LAXthr"

df = pd.read_parquet(
    "../data/processed/station_normals_temp_precip_sercc_archive.parquet"
).query(f"id == '{lax}'")

In [6]:
variable_names = {
    "maxt": "High",
    "avgt": "Average",
    "mint": "Low",
}

In [7]:
df["value_variable_clean"] = df["value_variable"].map(variable_names)

In [8]:
# Custom color scale
color_scale = alt.Scale(
    domain=["High", "Average", "Low"], range=["#f46d43", "#fdae61", "#fee08b"]
)

In [9]:
chart = (
    alt.Chart(df.query('value_variable_clean != "precip"'))
    .mark_line(interpolate="basis")
    .encode(
        x=alt.X("date:T", axis=alt.Axis(format="%B", tickCount=4), title=""),
        y=alt.Y(
            # "normal_value:Q", scale=alt.Scale(domain=[35, 85]), axis=alt.Axis(title="")
            "normal_value:Q",
            axis=alt.Axis(title="", tickCount=6),
        ),
        color=alt.Color("value_variable_clean:N", scale=color_scale, title=""),
    )
    .properties(width=400, height=600, title="Temperature normals at LAX ")
    .configure_legend(orient="top", symbolType="stroke")
)
chart

alt.Chart(...)

---

## Exports

#### JSON

In [10]:
df["date_normal"] = df["date"].str[-5:]

In [11]:
df.to_json(
    "../data/processed/station_normals_temp_precip_sercc_archive_lax.json",
    indent=4,
    orient="records",
    lines=False,
)